In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/sample_submission.csv
/kaggle/input/dataset/test_identity.csv
/kaggle/input/dataset/train_identity.csv
/kaggle/input/dataset/test_transaction.csv
/kaggle/input/dataset/train_transaction.csv


In [3]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.4.2 imbalanced-learn==0.12.2

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 12.1/12.1 MB 60.9 MB/s eta 0:00:00:00:010:01
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 258.0/258.0 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [4]:
!pip install imblearn
!pip install mlflow
!pip install optbinning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
import shap
import mlflow
import mlflow.sklearn

from optbinning import OptimalBinning
import warnings
warnings.filterwarnings("ignore")

   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 29.0/29.0 MB 37.4 MB/s eta 0:00:00:00:0100:01
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 6.3/6.3 MB 81.9 MB/s eta 0:00:00:00:0100:01
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 114.9/114.9 kB 6.0 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 85.0/85.0 kB 4.4 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 692.3/692.3 kB 28.2 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚

In [5]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.expand_frame_repr', False)

## Data Loading and Cleaning


In [6]:
import pandas as pd

# Load data
trans = pd.read_csv('/kaggle/input/dataset/train_transaction.csv')
ident = pd.read_csv('/kaggle/input/dataset/train_identity.csv')
# Merge on TransactionID
data = trans.merge(ident, on='TransactionID', how='left')

# Drop columns with >90% missing values
missing_frac = data.isnull().mean()
cols_to_drop = missing_frac[missing_frac > 0.90].index
data = data.drop(columns=cols_to_drop)

# Drop the TransactionID column
data = data.drop('TransactionID', axis=1)


## Train-Test Split

In [7]:
from sklearn.model_selection import train_test_split

y = data['isFraud']
X = data.drop('isFraud', axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

X_sample, _, y_sample, _ = train_test_split(X, y, train_size=0.5, random_state=42)

## Preprocessing and Feature Engineering

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class MissingValueFiller(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.numeric_cols = None
        self.categorical_cols = None
        self.medians = None

    def fit(self, X, y=None):
        numeric = X.select_dtypes(include=[np.number])
        categorical = X.select_dtypes(include=['object'])
        self.numeric_cols = numeric.columns.tolist()
        self.categorical_cols = categorical.columns.tolist()
        self.medians = numeric.median()
        return self

    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.numeric_cols] = X_copy[self.numeric_cols].fillna(self.medians)
        X_copy[self.categorical_cols] = X_copy[self.categorical_cols].fillna('missing')
        return X_copy


In [10]:
class WoEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=3):
        self.threshold = threshold
        self.woe_maps = {}
        self.large_cat = []

    def fit(self, X, y):
        categorical = X.select_dtypes(include=['object'])
        for col in categorical.columns:
            if categorical[col].nunique() > self.threshold:
                df = pd.DataFrame({'feature': X[col], 'target': y})
                cross = df.groupby(['feature', 'target']).size().unstack(fill_value=0)
                p1 = (cross.get(1, 0) + 0.5) / (y.sum() + 0.5)
                p0 = (cross.get(0, 0) + 0.5) / ((len(y) - y.sum()) + 0.5)
                self.woe_maps[col] = np.log(p1 / p0)
                self.large_cat.append(col)
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in self.large_cat:
            X_copy[col] = X_copy[col].map(self.woe_maps[col]).fillna(0)
        return X_copy


In [11]:
import shap
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

class AutoSHAPFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, model_class, model_params=None, threshold=0.95):
        self.model_class = model_class
        self.model_params = model_params if model_params is not None else {}
        self.threshold = threshold
        self.selected_features_idx = None

    def fit(self, X, y):
        # Train a lightweight model internally
        model = self.model_class(**self.model_params)
        model.fit(X, y)

        # Compute SHAP values
        import shap
        explainer = shap.Explainer(model, X)
        shap_values = explainer(X)

        # Calculate mean absolute SHAP values
        importance = np.abs(shap_values.values).mean(0)
        importance_df = pd.DataFrame({
            "feature_idx": np.arange(X.shape[1]),
            "importance": importance
        }).sort_values("importance", ascending=False)

        # Cumulative importance
        importance_df["cumulative"] = importance_df["importance"].cumsum() / importance_df["importance"].sum()

        # Select features
        self.selected_features_idx = importance_df[importance_df["cumulative"] <= self.threshold]["feature_idx"].tolist()
        return self

    def transform(self, X):
        return X[:, self.selected_features_idx]

    def get_feature_names_out(self, input_features=None):
        if input_features is not None:
            return [input_features[idx] for idx in self.selected_features_idx]
        else:
            return None

In [12]:
import numpy as np

# Identify numeric and categorical columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

small_cat = [col for col in categorical_cols if X_train[col].nunique() <= 3]

## Pipeline Construction

In [13]:
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

# Define column transformer: scale numeric and one-hot encode small cat
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown="ignore"), small_cat)
], remainder='passthrough')

In [14]:
RANDOM_STATE = 42

pipeline = Pipeline([
    ('undersampler', RandomUnderSampler(sampling_strategy=0.2, random_state=42)),
    ('missing', MissingValueFiller()),
    ('woe', WoEEncoder(threshold=3)),
    ('preprocessor', preprocessor),
    ('selector', AutoSHAPFeatureSelector(model_class=XGBClassifier, threshold=0.95)),
    ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE))
])

## Hyperparameter Tuning

In [16]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=2, shuffle=True, random_state=42)

In [25]:
from sklearn.model_selection import GridSearchCV

param_grid_md = {'classifier__max_depth': [10, 15]}
grid_search_md = GridSearchCV(
    pipeline, param_grid=param_grid_md, 
    scoring='roc_auc', cv=kfold, n_jobs=-1
)
grid_search_md.fit(X_sample, y_sample)

best_max_depth = grid_search_md.best_params_['classifier__max_depth']
print(f"Best max_depth: {best_max_depth}")

pipeline.set_params(classifier__max_depth=best_max_depth)

100%|===================| 61225/61362 [03:07<00:00]        

Best max_depth: 15


In [18]:
## kaggle getting rammed out so i know my best_max_depth was 15
pipeline.set_params(classifier__max_depth=15)

Pipeline(steps=[('undersampler',
                 RandomUnderSampler(random_state=42, sampling_strategy=0.2)),
                ('missing', MissingValueFiller()), ('woe', WoEEncoder()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['TransactionDT',
                                                   'TransactionAmt', 'card1',
                                                   'card2', 'card3', 'card5',
                                                   'addr1', 'addr2', 'dist1',
                                                   'C1', 'C2', 'C3', 'C4'...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=15, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=500, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [25]:
param_grid_lr = {'classifier__learning_rate': [0.2, 0.35, 0.5]}
grid_search_lr = GridSearchCV(
    pipeline, param_grid=param_grid_lr,
    cv=kfold, scoring='roc_auc', n_jobs=-1, verbose=1
)
grid_search_lr.fit(X_sample, y_sample)

best_learning_rate = grid_search_lr.best_params_['classifier__learning_rate']
print(f"Best learning_rate: {best_learning_rate}")

pipeline.set_params(classifier__learning_rate=best_learning_rate)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


100%|===================| 61222/61362 [03:01<00:00]        

Best learning_rate: 0.2


Pipeline(steps=[('undersampler',
                 RandomUnderSampler(random_state=42, sampling_strategy=0.2)),
                ('missing', MissingValueFiller()), ('woe', WoEEncoder()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['TransactionDT',
                                                   'TransactionAmt', 'card1',
                                                   'card2', 'card3', 'card5',
                                                   'addr1', 'addr2', 'dist1',
                                                   'C1', 'C2', 'C3', 'C4'...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=15, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [16]:
pipeline.set_params(classifier__learning_rate=0.2)

Pipeline(steps=[('undersampler',
                 RandomUnderSampler(random_state=42, sampling_strategy=0.2)),
                ('missing', MissingValueFiller()), ('woe', WoEEncoder()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['TransactionDT',
                                                   'TransactionAmt', 'card1',
                                                   'card2', 'card3', 'card5',
                                                   'addr1', 'addr2', 'dist1',
                                                   'C1', 'C2', 'C3', 'C4'...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [28]:
param_grid_ne = {'classifier__n_estimators': [500, 750 , 1000]}
grid_search_ne = GridSearchCV(
    pipeline, param_grid=param_grid_ne,
    cv=kfold, scoring='roc_auc', n_jobs=-1, verbose=1
)
grid_search_ne.fit(X_sample, y_sample)

best_n_estimators = grid_search_ne.best_params_['classifier__n_estimators']
print(f"Best n_estimators: {best_n_estimators}")

# Update pipeline with best n_estimators
pipeline.set_params(classifier__n_estimators=best_n_estimators)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


100%|===================| 61296/61362 [03:02<00:00]        

Best n_estimators: 500


Pipeline(steps=[('undersampler',
                 RandomUnderSampler(random_state=42, sampling_strategy=0.2)),
                ('missing', MissingValueFiller()), ('woe', WoEEncoder()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['TransactionDT',
                                                   'TransactionAmt', 'card1',
                                                   'card2', 'card3', 'card5',
                                                   'addr1', 'addr2', 'dist1',
                                                   'C1', 'C2', 'C3', 'C4'...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=15, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=500, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [17]:
pipeline.set_params(classifier__n_estimators=500)

Pipeline(steps=[('undersampler',
                 RandomUnderSampler(random_state=42, sampling_strategy=0.2)),
                ('missing', MissingValueFiller()), ('woe', WoEEncoder()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['TransactionDT',
                                                   'TransactionAmt', 'card1',
                                                   'card2', 'card3', 'card5',
                                                   'addr1', 'addr2', 'dist1',
                                                   'C1', 'C2', 'C3', 'C4'...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=500, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [19]:
pipeline.fit(X_train, y_train)

100%|===================| 99036/99180 [05:16<00:00]        

Pipeline(steps=[('undersampler',
                 RandomUnderSampler(random_state=42, sampling_strategy=0.2)),
                ('missing', MissingValueFiller()), ('woe', WoEEncoder()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['TransactionDT',
                                                   'TransactionAmt', 'card1',
                                                   'card2', 'card3', 'card5',
                                                   'addr1', 'addr2', 'dist1',
                                                   'C1', 'C2', 'C3', 'C4'...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=15, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=500, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

## Model Evaluation

In [20]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, classification_report
)

# best_model = grid.best_estimator_
best_model = pipeline

# assume `best_model` is already fitted (pipeline or grid.best_estimator_)
# 1) TRAIN METRICS
y_train_pred  = best_model.predict(X_train)
y_train_score = best_model.predict_proba(X_train)[:,1]

print("=== TRAIN SET REPORT ===")
print(classification_report(y_train, y_train_pred))

train_acc   = accuracy_score(y_train, y_train_pred)
train_prec  = precision_score(y_train, y_train_pred)
train_rec   = recall_score(y_train, y_train_pred)
train_f1    = f1_score(y_train, y_train_pred)
train_auc   = roc_auc_score(y_train, y_train_score)

print(f"Train Acc:   {train_acc:.4f}")
print(f"Train Prec:  {train_prec:.4f}")
print(f"Train Rec:   {train_rec:.4f}")
print(f"Train F1:    {train_f1:.4f}")
print(f"Train AUC:   {train_auc:.4f}")

fpr_train, tpr_train, _ = roc_curve(y_train, y_train_score)

# 2) TEST METRICS
y_test_pred  = best_model.predict(X_test)
y_test_score = best_model.predict_proba(X_test)[:,1]

print("\n=== TEST SET REPORT ===")
print(classification_report(y_test, y_test_pred))

test_acc   = accuracy_score(y_test, y_test_pred)
test_prec  = precision_score(y_test, y_test_pred)
test_rec   = recall_score(y_test, y_test_pred)
test_f1    = f1_score(y_test, y_test_pred)
test_auc   = roc_auc_score(y_test, y_test_score)

print(f"Test Acc:   {test_acc:.4f}")
print(f"Test Prec:  {test_prec:.4f}")
print(f"Test Rec:   {test_rec:.4f}")
print(f"Test F1:    {test_f1:.4f}")
print(f"Test AUC:   {test_auc:.4f}")

fpr_test, tpr_test, _ = roc_curve(y_test, y_test_score)

# 3) PLOT BOTH ROC CURVES OVERLAID
plt.figure(figsize=(8,6))
plt.plot(fpr_train, tpr_train, label=f"Train ROC (AUC={train_auc:.3f})")
plt.plot(fpr_test,  tpr_test,  label=f"Test  ROC (AUC={test_auc:.3f})")
plt.plot([0,1],[0,1],"k--", alpha=0.5)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves: Train vs Test")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("roc_train_vs_test.png")
plt.show()


=== TRAIN SET REPORT ===
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    455902
           1       0.80      1.00      0.89     16530

    accuracy                           0.99    472432
   macro avg       0.90      1.00      0.94    472432
weighted avg       0.99      0.99      0.99    472432

Train Acc:   0.9914
Train Prec:  0.8027
Train Rec:   1.0000
Train F1:    0.8906
Train AUC:   0.9995

=== TEST SET REPORT ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    113975
           1       0.73      0.80      0.76      4133

    accuracy                           0.98    118108
   macro avg       0.86      0.89      0.87    118108
weighted avg       0.98      0.98      0.98    118108

Test Acc:   0.9823
Test Prec:  0.7253
Test Rec:   0.7953
Test F1:    0.7587
Test AUC:   0.9714


## Logging on MlFlow

In [21]:
!pip install dagshub
!pip install mlflow

   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 260.1/260.1 kB 8.0 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 203.2/203.2 kB 9.8 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 74.3/74.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: graphql-core
    Found existing installation: graphql-core 3.2.6
    Uninstalling graphql-core-3.2.6:
      Successfully uninstalled graphql-core-3.2.6
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully uninstalled dacite-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.1

In [23]:
import dagshub
dagshub.init(repo_owner='losaberidzebadri', repo_name='IEEE-CIS-Fraud-Detection', mlflow=True)

‚ùó‚ùó‚ùó AUTHORIZATION REQUIRED ‚ùó‚ùó‚ùó

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=99228b2e-e760-4488-95d6-e8b561800e22&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=86d4960c87f695789c0b9ea97235407f648a4bf1e502cc7e5f39a2714d157d16




Accessing as losaberidzebadri

Initialized MLflow to track repo "losaberidzebadri/IEEE-CIS-Fraud-Detection"

Repository losaberidzebadri/IEEE-CIS-Fraud-Detection initialized!

In [24]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("XGBClassifierTraining")

# Start MLflow run
with mlflow.start_run(run_name="XGBClassifier_BestParams"):
    # mlflow.log_param("max_depth", best_max_depth)
    mlflow.log_param("max_depth", 15) ## already know this from previous cv
    mlflow.log_param("learning_rate", 0.2)
    mlflow.log_param("n_estimators", 500)
    
    # log train metrics
    mlflow.log_metric("train_accuracy",   train_acc)
    mlflow.log_metric("train_precision",  train_prec)
    mlflow.log_metric("train_recall",     train_rec)
    mlflow.log_metric("train_f1",         train_f1)
    mlflow.log_metric("train_auc",        train_auc)
    # log test metrics
    mlflow.log_metric("test_accuracy",    test_acc)
    mlflow.log_metric("test_precision",   test_prec)
    mlflow.log_metric("test_recall",      test_rec)
    mlflow.log_metric("test_f1",          test_f1)
    mlflow.log_metric("test_auc",         test_auc)
    # log the ROC plot
    mlflow.log_artifact("roc_train_vs_test.png")
    # log the model itself
    mlflow.sklearn.log_model(best_model, "final_model")



2025/04/27 20:08:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


üèÉ View run XGBClassifier_BestParams at: https://dagshub.com/losaberidzebadri/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/1/runs/5fa2b1b991e949f3a52fa72b0e90d980
üß™ View experiment at: https://dagshub.com/losaberidzebadri/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/1
